In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression, Lasso
import mlflow

In [4]:
mlflow.set_tracking_uri('sqlite:///mlflow.db') # set the database
mlflow.set_experiment('my-brand-new-experiment') # create the experiment if not exists

INFO: 'my-brand-new-experiment' does not exist. Creating a new experiment


In [42]:
def read_dataframe(path):
  df = pd.read_parquet(path)
  df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
  df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
  df['duration'] = (df.lpep_dropoff_datetime - df.lpep_pickup_datetime).dt.seconds / 60

  df = df[((df.duration >=1) & (df.duration <=60))]
  return df

In [31]:
df_train = read_dataframe('green_tripdata_2021-01.parquet')
df_val = read_dataframe('green_tripdata_2021-02.parquet')

In [32]:
len(df_train), len(df_val)

(73908, 61921)

In [33]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df_train[categorical] = df_train[categorical].astype(str)
df_val[categorical] = df_val[categorical].astype(str)

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

valid_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(valid_dicts)

In [43]:
target = 'duration'

y_train = df_train[target].values
y_val = df_val[target].values

In [49]:
with mlflow.start_run():

            mlflow.set_tag("developer", "patrick") 
            mlflow.log_param("train-data-path", "green_trip_data_2021-01.csv")
            mlflow.log_param("train-val-path", "green_trip_data_2021-02.csv")

            alpha = 0.005
            mlflow.log_param("alpha", alpha)

            lr = Lasso(alpha)
            lr.fit(X_train, y_train)

            y_pred = lr.predict(X_val)

            rmse = mean_squared_error(y_val, y_pred, squared=False)
            mlflow.log_param("rmse", rmse)

In [48]:
for alpha in [0.1, 0.01, 0.001]:

    with mlflow.start_run():

            mlflow.set_tag("developer", "patrick") 
            mlflow.log_param("train-data-path", "green_trip_data_2021-01.csv")
            mlflow.log_param("train-val-path", "green_trip_data_2021-02.csv")

            mlflow.log_param("alpha", alpha)

            lr = Lasso(alpha)
            lr.fit(X_train, y_train)

            y_pred = lr.predict(X_val)

            rmse = mean_squared_error(y_val, y_pred, squared=False)
            mlflow.log_param("rmse", rmse)